In [1]:
#this notebook is to be executed on google colab
from google.colab import drive
drive.mount('/content/d')

Drive already mounted at /content/d; to attempt to forcibly remount, call drive.mount("/content/d", force_remount=True).


In [2]:
!unzip /content/d/My\ Drive/raw_pictures.zip

Archive:  /content/d/My Drive/raw_pictures.zip
replace content/trying/644405char3.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
!mkdir faces
!mkdir faces/smaller_than_118x118
!mkdir faces/118x118_to_127x127
!mkdir faces/128x128_to_175x175
!mkdir faces/bigger_than_175x175

mkdir: cannot create directory ‘faces’: File exists
mkdir: cannot create directory ‘faces/smaller_than_118x118’: File exists
mkdir: cannot create directory ‘faces/118x118_to_127x127’: File exists
mkdir: cannot create directory ‘faces/128x128_to_175x175’: File exists
mkdir: cannot create directory ‘faces/bigger_than_175x175’: File exists


In [4]:
!pip install wget
import cv2
import sys
import os.path
import matplotlib.pyplot as plt 
import wget

In [5]:
wget.download('https://raw.githubusercontent.com/nagadomi/lbpcascade_animeface/master/lbpcascade_animeface.xml')

'lbpcascade_animeface (1).xml'

In [6]:
cascade_file = "/content/lbpcascade_animeface.xml"
cascade = cv2.CascadeClassifier(cascade_file)
def detect(image):
    if not os.path.isfile(cascade_file):
        raise RuntimeError("%s: not found" % cascade_file)

    cascade = cv2.CascadeClassifier(cascade_file)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    
    faces = cascade.detectMultiScale(gray,
                                     # detector options
                                     scaleFactor = 1.1,
                                     minNeighbors = 5,

                                    minSize = (24, 24))
    return faces

In [7]:
def zoom_out(faces):
  new_faces = []
  for (x,y,w,h) in faces: 
    x = x + w/2
    y = y + h/2
    w*= 2 
    h*= 2
    x-=w/2
    y-=h/2
    
    new_faces.append((int(x), int(y), int(w),int(h)))
  return new_faces


In [8]:
def cropper(img, faces): 
  cropped_faces = []
  for (x,y,w,h) in faces:
    x_1 = max(0,x)
    y_1 = max(0,y)
    x_2 = min(x+w, len(img[1]))
    y_2 = min(y+h, len(img[0]))
    cropped_faces.append(img[y_1:y_2, x_1:x_2])
  return cropped_faces

In [9]:
def extract_faces(dir):
  image = cv2.imread(dir, cv2.IMREAD_COLOR)
  faces = detect(image)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  faces = zoom_out(faces)
  faces = cropper(image , faces)
  return faces


In [10]:
import os
dir = '/content/content/trying'
images = os.listdir(dir)

In [11]:
from PIL import Image
import numpy as np
processed = 0
total = len(images)
def img_to_faces(img_name, save_dir): 
   faces = extract_faces('{}/{}'.format(dir,img_name))
   name = img_name.split('.')
   counter = 0
   for face in faces: 
     w = len(face[0])
     h = len(face[1])
     img = Image.fromarray(face)
     if w < 118 or h< 118: 
       img.save("{}/smaller_than_118x118/{}_face{}.jpg".format(save_dir,name[0],counter ))
     elif w < 127 or h < 127:
       img.save("{}/118x118_to_127x127/{}_face{}.jpg".format(save_dir,name[0],counter ))
     elif w < 175 or h < 175:
       img.save("{}/128x128_to_175x175/{}_face{}.jpg".format(save_dir,name[0],counter ))
     else:
       img.save("{}/bigger_than_175x175/{}_face{}.jpg".format(save_dir,name[0],counter ))
     counter += 1
   global processed
   processed += 1

In [12]:
failed = []
def imgs_to_faces(images , save_dir): 
  global failed
  for image in images:
    try:
     img_to_faces(image, save_dir)
    except Exception as err: 
      failed.append(image)
      print('error on image {}'.format(image))
      print(type(err))
      print(err.args)
      print(err)

In [13]:
import time
import random
import threading
def multi_thread_process(num_threads, images, save_dir):
  random.shuffle(images)
  threads = [] 
  img_per_thread =int( len(images) / num_threads)
  starting = time.time()
  for i in range( (num_threads)):
    thread = threading.Thread( target = imgs_to_faces, args = (images[i*img_per_thread:(i+1)*img_per_thread], save_dir))
    threads.append(thread)
    thread.start()
  while True:
     time.sleep(1)
     finished = True
     sequence_finished = True
     for thread in threads:
       sequence_finished &=  not thread.is_alive()
     if(sequence_finished):
      print('processing completed {}: processed'.format(processed))
      break
     else:
        print('{} processed {} remaining {}% completed'.format(processed, total - processed, processed / total * 100))


In [14]:
save_dir = "/content/faces"
processed = 0
multi_thread_process(100, images,save_dir)

611 processed 51963 remaining 1.1621714155285883% completed
659 processed 51915 remaining 1.2534712975995739% completed
702 processed 51872 remaining 1.3352607752881651% completed
759 processed 51815 remaining 1.4436793852474605% completed
797 processed 51777 remaining 1.5159584585536579% completed
869 processed 51705 remaining 1.6529082816601364% completed
902 processed 51672 remaining 1.715676950583939% completed
977 processed 51597 remaining 1.858333016319854% completed
997 processed 51577 remaining 1.8963746338494312% completed
1068 processed 51506 remaining 2.0314223760794308% completed
1102 processed 51472 remaining 2.0960931258797126% completed
1150 processed 51424 remaining 2.187393007950698% completed
1204 processed 51370 remaining 2.290105375280557% completed
1229 processed 51345 remaining 2.3376573971925287% completed
1309 processed 51265 remaining 2.489823867310838% completed
1339 processed 51235 remaining 2.546886293605204% completed
1387 processed 51187 remaining 2.638186

In [15]:
!zip -r /content/getchu_faces.zip /content/faces

Streaming output truncated to the last 5000 lines.
  adding: content/faces/bigger_than_175x175/704425char1_face0.jpg (deflated 3%)
  adding: content/faces/bigger_than_175x175/861010char1_face0.jpg (deflated 2%)
  adding: content/faces/bigger_than_175x175/805127char3_face0.jpg (deflated 3%)
  adding: content/faces/bigger_than_175x175/540485char0_face0.jpg (deflated 2%)
  adding: content/faces/bigger_than_175x175/897570char0_face0.jpg (deflated 2%)
  adding: content/faces/bigger_than_175x175/931387char26_face0.jpg (deflated 3%)
  adding: content/faces/bigger_than_175x175/884640char0_face0.jpg (deflated 3%)
  adding: content/faces/bigger_than_175x175/360305char10_face0.jpg (deflated 5%)
  adding: content/faces/bigger_than_175x175/787537char7_face0.jpg (deflated 5%)
  adding: content/faces/bigger_than_175x175/669310char7_face0.jpg (deflated 4%)
  adding: content/faces/bigger_than_175x175/283705char0_face0.jpg (deflated 2%)
  adding: content/faces/bigger_than_175x175/245345char0_face0.jpg (